## hdf5

In [1]:
from pathlib import Path
import numpy as np
import h5py
import json

In [2]:
filename_1 = Path("C:/LINHC/VersucheDBs/Trelleborg/2021-01-27-V24/2_Phase_A-D/20210127_Phase_A-D_LEM1.h5")

h51 = h5py.File(filename_1, "r")

print("ATTR:")
print(h51.attrs.keys())

print("Keys:")
print(h51.keys())

# frequency index
fidx = 20

ATTR:
<KeysViewHDF5 ['name_lookup', 'name_rev_lookup']>
Keys:
<KeysViewHDF5 ['S11_LEM1', 'S21_P1_LEM1', 'S21_P2_LEM1', 'S22_LEM1', 'hub1', 'hub2', 'ip1k1', 'ip1k2', 'ip2k1', 'ip2k2', 'k1at', 'k1f', 'k1p', 'k1t', 'k2at', 'k2f', 'k2p', 'k2t', 'phi', 'pos1', 'pos2', 'shub1', 'shub2', 'sk1at', 'sk1f', 'sk1p', 'sk1t', 'sk2at', 'sk2f', 'sk2p', 'sk2t', 'spd1', 'spd2', 'sphi', 'spos1', 'spos2', 'sspd1', 'sspd2', 'stdf', 't', 'tdf']>


In [6]:
h51['pos1'][:20000].shape

(20000,)

In [10]:
# truncate to first 100000 points
file = Path("C:/LINHC/VersucheDBs/Trelleborg/2021-01-27-V24/2_Phase_A-D/20210127_Phase_A-D_LEM1_100000.h5")

with h5py.File(file, "w") as f:
    f.attrs["name_lookup"] = h51.attrs["name_lookup"]
    f.attrs["name_rev_lookup"] = h51.attrs["name_rev_lookup"]
    for key in h51.keys():
        f.create_dataset(key, data = h51[key][:100000][:])
    print("ATTR:")
    print(h51.attrs.keys())
    print("Keys:")
    print(h51.keys())

ATTR:
<KeysViewHDF5 ['name_lookup', 'name_rev_lookup']>
Keys:
<KeysViewHDF5 ['S11_LEM1', 'S21_P1_LEM1', 'S21_P2_LEM1', 'S22_LEM1', 'hub1', 'hub2', 'ip1k1', 'ip1k2', 'ip2k1', 'ip2k2', 'k1at', 'k1f', 'k1p', 'k1t', 'k2at', 'k2f', 'k2p', 'k2t', 'phi', 'pos1', 'pos2', 'shub1', 'shub2', 'sk1at', 'sk1f', 'sk1p', 'sk1t', 'sk2at', 'sk2f', 'sk2p', 'sk2t', 'spd1', 'spd2', 'sphi', 'spos1', 'spos2', 'sspd1', 'sspd2', 'stdf', 't', 'tdf']>


In [11]:
f.close()


In [ ]:
# save the file as hdf5

file = r'C:\LINHC\VersucheDBs\Trelleborg\test\training.h5'

with h5py.File(file, 'w') as f:
    setup = f.create_group("setup")
    liview = f.create_group("liview")
    ref = f.create_group("ref")
    
    liview.create_dataset('s21',data=np.array(s21).reshape(121,-1))
    
    for feature in features_setup:
        tmp = np.array(eval(feature)).reshape(-1)
        setup.create_dataset(feature,data=tmp)
        
    for gt in ground_truths:
        tmp = np.array(eval(gt)).reshape(-1)
        ref.create_dataset(gt,data=tmp)
        
    feature_lookup={}
    for j in range(242):
        if j%2 == 0:
            key = 's21-'+str(j//2)+'-real'
        else:
            key = 's21-'+str(j//2)+'-imag'
        feature_lookup[key] = j
    
    i = 242
    for key in setup.keys():
        feature_lookup[key] = i
        i += 1

    f.attrs["feature_lookup"] = json.dumps(feature_lookup)     
    
    f.attrs["name_lookup_rev"] = h51.attrs["name_rev_lookup"]

In [ ]:
h51.close()

In [ ]:
## Read all the training data and labels from h5 files
file = r'C:\LINHC\VersucheDBs\Trelleborg\test\training.h5'
f = h5py.File(file, 'r')
print("ATTR:")
print(f.attrs.keys())
print("Keys:")
print(f.keys())

setup = f['setup']
s21 = f['liview']['s21']

dataset = []
for i in range(121):
    dataset.append(s21[i].real)
    dataset.append(s21[i].imag)
for value in setup.values():
    dataset.append(np.asarray(value))
    
X = np.asarray(dataset).T
y = np.asarray(f['ref']['pos1'])
print("X:")
print(X.shape)
print("y:")
print(y.shape)

## pickle

In [28]:
import pickle

In [30]:
data = {
    "KEY 1": VALUE 1,
    "KEY 2" : VALUE 2,
    ....
}
pickle.dump(data, open( "FILENAME", "wb" ) )

In [27]:
pickle.load( open( "FILENAME", "rb" ) )

<bound method File.close of <HDF5 file "test.hdf5" (mode r+)>>